In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import plotly_express as px
import tqdm.notebook as tqdm

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.preprocessing import deseq2_norm
from pydeseq2.preprocessing import make_counts_long
from pydeseq2.preprocessing import rle_norm
from pydeseq2.utils import load_example_data

In [4]:
counts_df = load_example_data(
    modality="raw_counts",
    dataset="synthetic",
    debug=False,
)

clinical_df = load_example_data(
    modality="clinical",
    dataset="synthetic",
    debug=False,
)

# specify plate group
clinical_df["shortplate"] = [1, 2] * 50
clinical_q1 = clinical_df[clinical_df["shortplate"] == 1]
clinical_q2 = clinical_df[clinical_df["shortplate"] == 2]

counts_q1 = counts_df.loc[clinical_q1.index, :].iloc[:, :5]
counts_q2 = counts_df.loc[clinical_q2.index, :].iloc[:, 5:]

counts_sparse = pd.concat([counts_q1, counts_q2])

# Load instances
dds_sparse = DeseqDataSet(
    counts_sparse,
    clinical_df,
    design_factors="condition",
    plate_group="shortplate",
    refit_cooks=False,
    n_cpus=8,
    silent=True,
)

# Fit size factors
long_counts = make_counts_long(counts_df, clinical_df, "shortplate")


dds_sparse.set_size_factors(
    rle_norm(
        df_long=long_counts,
        response="ct",
        un_norm=[],
        gene_col="gene",
        sample_col="sample",
        gene_filter_stringency="loose",
        log_func=np.log,
    )[1]
)

# Pipeline Run
dds_sparse.fit_genewise_dispersions()
dds_sparse.fit_dispersion_trend()
dds_sparse.fit_dispersion_prior()
dds_sparse.fit_MAP_dispersions()
dds_sparse.fit_LFC()
dds_sparse.calculate_cooks()
dds_sparse.refit()

# Stats
stat_sparse = DeseqStats(
    dds_sparse, n_cpus=8, cooks_filter=True, independent_filter=False, silent=True
)

stat_sparse.summary()
stat_sparse.lfc_shrink()

stat_sparse.results_df["subset"] = "sparse"

results_expt5 = [stat_sparse.results_df]

stat_sparse.results_df

Entries removed: 35


/Users/ikristanto/opt/miniconda3/envs/gridx/lib/python3.8/site-packages/pandas/core/base.py:880: RuntimeWarning: divide by zero encountered in log
  new_values = map_f(values, mapper)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,subset
gene1,6.541087,0.993509,0.360911,3.451976,0.000556,0.005565,sparse
gene2,20.032305,0.298775,0.187595,1.982887,0.047380,0.118450,sparse
gene3,6.039382,-0.327726,0.335814,-1.738741,0.082080,0.131737,sparse
gene4,124.521236,-0.382826,0.141044,-3.044752,0.002329,0.011644,sparse
gene5,25.355235,0.470588,0.221610,2.659108,0.007835,0.026116,sparse
gene6,4.044103,-0.047598,0.317736,-0.352051,0.724800,0.724800,sparse
gene7,31.326275,0.106094,0.226259,0.700056,0.483892,0.537658,sparse
gene8,46.011955,-0.236528,0.173060,-1.683822,0.092216,0.131737,sparse
gene9,38.497347,-0.222489,0.156218,-1.696876,0.089720,0.131737,sparse
gene10,12.369936,0.193854,0.253602,1.193108,0.232827,0.291034,sparse


In [5]:
# controls
dds_q1 = DeseqDataSet(
    counts_q1,
    clinical_q1,
    design_factors="condition",
    plate_group="shortplate",
    refit_cooks=False,
    n_cpus=8,
    silent=True,
)

dds_q2 = DeseqDataSet(
    counts_q2,
    clinical_q2,
    design_factors="condition",
    plate_group="shortplate",
    refit_cooks=False,
    n_cpus=8,
    silent=True,
)


dds_q1.size_factors = dds_sparse.size_factors.loc[clinical_q1.index]
dds_q2.size_factors = dds_sparse.size_factors.loc[clinical_q2.index]


# Pipeline Run
for _dds, q in zip([dds_q1, dds_q2], ["q1", "q2"]):
    _dds.fit_genewise_dispersions()
    _dds.fit_dispersion_trend()
    _dds.fit_dispersion_prior()
    _dds.fit_MAP_dispersions()
    _dds.fit_LFC()
    _dds.calculate_cooks()
    _dds.refit()

    _stat = DeseqStats(
        _dds, n_cpus=8, cooks_filter=False, independent_filter=False, silent=True
    )

    _stat.summary()
    _stat.results_df["subset"] = q

    results_expt5.append(_stat.results_df)

In [6]:
results_expt5 = pd.concat(results_expt5)

for metric in ["baseMean", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj"]:
    px.line(
        results_expt5.reset_index(),
        x="index",
        y=metric,
        color="subset",
        markers=True,
        title=f"Comparison of {metric.upper()} across N-split data.",
    ).show()